# 4. Analizando la matriz de Hilbert

El objetivo de este Notebook es analizar la matriz de Hilbert, una matriz que suele usarse en problemas de interpolación y aproximación númerica.

La Matriz de Hilbert se define de la siguiente manera: `Hn(i, j) = 1/i+j−1`



## Importando Liberías

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import plotly_express as px
from scipy.linalg import hilbert
from scipy.sparse.linalg import cg


## 4.1. Analizando la Condición de la Matriz de Hilbert

#### Generando la Matriz de Hilbert

Se propone la siguiente función para el cálculo, sin embargo, también se podría usar la función `hilbert` de la librería `scipy`.

In [2]:
# Creando función para calcular la matriz de Hilbert

def generate_hilbert_matrix(N):

    hilbert_matrix = np.array([[1 / (i + j - 1) for j in range(1, N + 1)] for i in range(1, N + 1)])
    return hilbert_matrix

In [5]:
# Ejemplo de uso
generate_hilbert_matrix(3)

array([[1.        , 0.5       , 0.33333333],
       [0.5       , 0.33333333, 0.25      ],
       [0.33333333, 0.25      , 0.2       ]])

In [5]:
# Validando reusltados con función scipy.linalg.hilbert

hilbert(3)

array([[1.        , 0.5       , 0.33333333],
       [0.5       , 0.33333333, 0.25      ],
       [0.33333333, 0.25      , 0.2       ]])

#### Creando función para calcular la covarianza de la Matriz

In [6]:
# Creando función para calcular la covarianza de la matriz de hilbert

def calculate_covariance_matrix(hilbert_matrix):

    return np.cov(hilbert_matrix, rowvar=False)

In [7]:
# Ejemplo de uso
calculate_covariance_matrix(generate_hilbert_matrix(3))

array([[0.12037037, 0.04398148, 0.02314815],
       [0.04398148, 0.0162037 , 0.00856481],
       [0.02314815, 0.00856481, 0.00453704]])

#### Simulando `k` datos normales con la Matriz de Covarianza `Hn`

In [8]:
def generate_normal_data(N, num_samples):
    mean = np.zeros(N)
    hilbert_matrix = generate_hilbert_matrix(N)
    covariance_matrix = calculate_covariance_matrix(hilbert_matrix)
    samples = np.random.multivariate_normal(mean, covariance_matrix, num_samples)  #Genera datos con distribución normal con matriz de covarianza Hn
    cov_samples=np.cov(samples) #Estima la matriz de covarianza desde los datos simulados
    return samples,cov_samples

In [9]:
#Ejemplo de uso para k=1000
data,cov_data=generate_normal_data(3, 1000)

print("\nDatos simulados:")
print("Shape de datos generados: ",data.shape)
print("Media de datos generados: ",data.mean())
display(data)

print("Matriz de covarianzas de los datos simulados:")
print("Shape de la matriz de covarianza: ",cov_data.shape)
display(cov_data)


Datos simulados:
Shape de datos generados:  (1000, 3)
Media de datos generados:  -0.01588337198481874


array([[ 0.23231787,  0.09194137,  0.05032131],
       [ 0.27623717,  0.09567312,  0.04891478],
       [-0.45227742, -0.17635835, -0.09585894],
       ...,
       [ 0.01410923,  0.00900347,  0.00579185],
       [ 0.19528235,  0.06180147,  0.02991294],
       [-0.08980439, -0.03013958, -0.01513122]])

Matriz de covarianzas de los datos simulados:
Shape de la matriz de covarianza:  (1000, 1000)


array([[ 9.09342291e-03,  1.14441546e-02, -1.78249800e-02, ...,
         3.94021848e-04,  8.36024076e-03, -3.76507414e-03],
       [ 1.14441546e-02,  1.44108650e-02, -2.24344979e-02, ...,
         4.93802256e-04,  1.05308456e-02, -4.74166142e-03],
       [-1.78249800e-02, -2.24344979e-02,  3.49409373e-02, ...,
        -7.71963018e-04, -1.63896072e-02,  7.38095320e-03],
       ...,
       [ 3.94021848e-04,  4.93802256e-04, -7.71963018e-04, ...,
         1.75936827e-05,  3.59895910e-04, -1.62320074e-04],
       [ 8.36024076e-03,  1.05308456e-02, -1.63896072e-02, ...,
         3.59895910e-04,  7.69684432e-03, -3.46522721e-03],
       [-3.76507414e-03, -4.74166142e-03,  7.38095320e-03, ...,
        -1.62320074e-04, -3.46522721e-03,  1.56020367e-03]])

## Analizando gráficamente la condición de la matriz de Hilbert

In [10]:
# Definiendo función para gráficar la condición de la Matriz de Hilbert para N elementos:
def plot_condition_of_Hilbert_matrix(N):
    n_values = np.arange(2, N+1)  

    # Initialize an empty list to store condition numbers
    condition_numbers = []

    # Calculate the condition number for each n
    for n in n_values:
        hilbert_matrix = generate_hilbert_matrix(n)
        condition_number = np.linalg.cond(hilbert_matrix,p=np.inf)
        condition_numbers.append(condition_number)

    # Create a DataFrame with n and condition numbers
    data = {'n': n_values, 'Condition Number': condition_numbers}
    df = pd.DataFrame(data)

    # Create the plot using Plotly Express
    fig = px.line(df, x='n', y='Condition Number', title=f'Condition Number of Hilbert Matrix vs. n for n={N}')
    fig.update_layout(xaxis_title='n', yaxis_title='Condition Number', xaxis_tickformat='d')#, yaxis_tickformat='d')

    fig.show()

In [13]:
#Usando función
plot_condition_of_Hilbert_matrix(15)

#### **Observación**: Es evidente que a medida que aumenta el valor de `n` en la **Matriz de Hilbert**, aumenta su condición y, por lo tanto, aumenta su **Cota superior de error relativo**. Esto significa que la matriz tiende a ser mal condicionada a medida que aumenta `n`, por lo tanto, se vuelve cada vez mas sensible a pequeñas perturbaciones en los datos y puede causar serios problemas en la precisión de su estimación numérica. 

## 4.2. Estimando valores de X usando la Matriz de Hilbert

### Definiendo vector x

In [14]:
x= np.arange(1, 16)
x

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15])

#### Generando la Matriz de Hilbert para n=15

In [15]:
H15=generate_hilbert_matrix(15)
H15

array([[1.        , 0.5       , 0.33333333, 0.25      , 0.2       ,
        0.16666667, 0.14285714, 0.125     , 0.11111111, 0.1       ,
        0.09090909, 0.08333333, 0.07692308, 0.07142857, 0.06666667],
       [0.5       , 0.33333333, 0.25      , 0.2       , 0.16666667,
        0.14285714, 0.125     , 0.11111111, 0.1       , 0.09090909,
        0.08333333, 0.07692308, 0.07142857, 0.06666667, 0.0625    ],
       [0.33333333, 0.25      , 0.2       , 0.16666667, 0.14285714,
        0.125     , 0.11111111, 0.1       , 0.09090909, 0.08333333,
        0.07692308, 0.07142857, 0.06666667, 0.0625    , 0.05882353],
       [0.25      , 0.2       , 0.16666667, 0.14285714, 0.125     ,
        0.11111111, 0.1       , 0.09090909, 0.08333333, 0.07692308,
        0.07142857, 0.06666667, 0.0625    , 0.05882353, 0.05555556],
       [0.2       , 0.16666667, 0.14285714, 0.125     , 0.11111111,
        0.1       , 0.09090909, 0.08333333, 0.07692308, 0.07142857,
        0.06666667, 0.0625    , 0.05882353, 

#### Hallando `b`, donde `b`= `H15` * `x`

In [16]:
b=H15@x
b

array([15.        , 12.61927101, 11.12089495, 10.01467577,  9.1423747 ,
        8.42796838,  7.82784777,  7.31430725,  6.86852505,  6.47709069,
        6.13010076,  5.82003392,  5.5410471 ,  5.28851531,  5.05871941])

#### Resolviendo el sistema utilizando la forma `x`=`Inv_H15` * `b`

##### Calculando `Inv_H15`

In [17]:
I_H15=np.linalg.inv(H15)
I_H15

array([[ 1.58102325e+02, -1.23889847e+04,  3.15964881e+05,
        -3.86891496e+06,  2.67618553e+07, -1.13312251e+08,
         3.03708680e+08, -5.10569864e+08,  4.93534545e+08,
        -1.71580270e+08, -1.43728572e+08,  1.60546278e+08,
        -2.04394795e+07, -3.37170153e+07,  1.23612882e+07],
       [-1.24139480e+04,  1.29893443e+06, -3.73711517e+07,
         4.90142708e+08, -3.55249071e+09,  1.55982784e+10,
        -4.31721807e+10,  7.49844614e+10, -7.54267804e+10,
         2.83167578e+10,  2.32445585e+10, -3.12735169e+10,
         1.04566898e+10,  1.57293115e+09, -1.20276845e+09],
       [ 3.18072012e+05, -3.75526038e+07,  1.15731114e+09,
        -1.59009456e+10,  1.19424887e+11, -5.40541259e+11,
         1.53971946e+12, -2.75663156e+12,  2.87538197e+12,
        -1.14181680e+12, -9.68567378e+11,  1.49246049e+12,
        -7.14620031e+11,  9.37033423e+10,  1.62678082e+10],
       [-3.92638714e+06,  4.96768669e+08, -1.60423758e+10,
         2.28401603e+11, -1.76741998e+12,  8.22121729

In [19]:
x_estimated=I_H15@b
x_estimated

array([-4.50581565e+00,  2.00421654e+02, -6.07346191e+02,  5.78531250e+02,
       -1.25225000e+03,  4.75100000e+03, -7.84800000e+03,  6.35200000e+03,
       -2.33600000e+03, -6.40000000e+01,  3.84000000e+02, -1.44000000e+02,
        5.60000000e+01, -2.00000000e+00,  1.92656250e+01])

#### Resolviendo el sistema utilizando la función de numpy `loinalg.solve`

In [24]:
# Resolver el sistema lineal x = H^(-1) * b
x_estimated = np.linalg.solve(H15, b)

# Imprimir la solución estimada
x_estimated

array([  0.99999951,   2.00007576,   2.99712233,   4.04722415,
         4.58549494,   8.14944695,   0.18557238,  20.63897678,
        -0.6744124 ,  -2.16371952,  53.6820964 , -42.3648982 ,
        51.00101332,  -0.41395869,  17.32996634])

### **Conclusión:** Se puede notar que independientemende del método utilizado, la precisión de la estimación de los valores no tiene ningún sentido respecto a los valores reales. Esto es coherente con la gráfica del punto anterior, donde se evidención que entre más aumenta n, menor es la precisión de la estimación porque aumenta la condición.

## Alternativas para resolver el problema observado

### **1. Regularización**: La regularicazión (ridge) suma una pequeña cantidad (`alpha`) de la matriz identidad a la matriz de HIlbert antes de resolver el sistema de ecuaciones lineales.

In [27]:
# Regularización (Ridge Regression)
alpha = 1e-9  # Parámetro de regularización (ajustable)
A = H15 + alpha * np.identity(15)
x_ridge = np.linalg.solve(A, b)
x_ridge

array([ 0.99999973,  2.00001699,  2.9997758 ,  4.00101915,  4.9984947 ,
        5.99944425,  7.00166846,  8.00166572,  8.99957563,  9.99764421,
       10.99776733, 12.00015367, 13.00308267, 14.00326435, 14.99642161])

#### Es destacable la mejora en los resultados de la estimación cuando se aplica Regularización, en este caso, todos los valores estimados son casi iguales a los valores reales.

### **2. Gradiente conjugado o Gradiente descendiente:** Es un algoritmo iterativo utilizado para resolver sistemas lineales de ecuaciones en matrices simétricas y positivas definidas. Este método es particularmente efectivo en matrices mal condicionadas y es más eficiente que los métodos directos, como la inversión de matriz, en tales casos.

In [28]:
x_approx, _ = cg(H15, b, tol=1e-6)  # Ajustar la tolerancia según sea necesario

# Imprimir la solución aproximada
x_approx

array([ 1.01364205,  1.80885355,  3.46770725,  4.01629767,  4.69368057,
        5.65677151,  6.80213702,  8.01231518,  9.2039909 , 10.32721728,
       11.35597582, 12.27964888, 13.0969328 , 13.81184227, 14.43119377])

#### Se puede notar que este método mejora la precisión de la estimación en algunas observaciones, pero afecta algunas que eran precisas antes de esta transformación, por ejemplo, la segunda estimación pasa de 2 a 1.8.

#### Por lo tanto, en este caso, es preferible utilizar la Regularización `ridge regression` para resolver el problema de precisión en la estimación numérica a medida que aumenta `n` en la Matriz de Hilbert.